In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

#데이터 전처리 방식 지정.
transform = transforms.Compose([
    transforms.ToTensor(), # To transfer data to Tensor type
    transforms.Normalize(mean=(0.5,), std=(0.5,)) # pixel value [0~1] -> [-1 ~ 1]
    #평균값 0.5 표준편차 0.5로 적용해서 정규화시키겠다.
])


mnist = datasets.MNIST(root='data', download=True, transform=transform)
#transform 옵션은 뭘까?

dataloader = DataLoader(mnist, batch_size=60, shuffle=True)

Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw


Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw
Processing...


..\torch\csrc\utils\tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


Done!


In [3]:
#생성자 (Generator)
#랜덤벡터 z를 입력으로 받아 가짜 이미지를 출력
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(in_feature=100, out_features=256),
            nn.LeakyReLU(0.2, inplace=True),#activity funciton
            nn.Linear(in_feature=256, out_features=512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_feature=512, out_features=1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_feature=1024, out_features=28*28),
            nn.Tanh()) #출력값을 픽셀값의 범위인 -1 ~ 1 사이로 만들어주기 위해 사용
        #LeakyReLU : 각 뉴런의 출력값이 0보다 높으면 놔두고, 
        #            0보다 낮으면 정해진 작은 숫자를 곱하는 함수
        
        # (batch_size * 100) 크기의 랜덤 벡터를 받아
        #이미지를 (batch_size * 1 * 28 * 28) 크기로 출력한다.
    def forward(self,inputs):
        return self.main(inputs).view(-1,1,28,28)

In [4]:
#구분자(Discriminator)
#이미지를 입력으로 받아 이미지가 진짜인지 가짜인지 출력
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(inplace=True),
            nn.Linear(in_features=1024, out_features=512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(inplace=True),
            nn.Linear(in_features=512, out_features=256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(inplace=True),
            nn.Linear(in_features=256, out_features=1),
            nn.Sigmoid())
    
    # (batch_size x 1 x 28 x 28) 크기의 이미지를 받아
    # 이미지가 진짜일 확률을 0~1 사이로 출력한다.
    
    def forward(self, inputs):
        inputs = inputs.view(-1, 28*28)
        return self.main(inputs)

In [ ]:
# 생성자 , 구분자 객체 생성
G = Generator()
D = Discriminator()
